<a href="https://colab.research.google.com/github/Typing784/tutorials/blob/master/HF_Note8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install datasets evaluate transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.5 MB/s eta 0:00:00


In [77]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    result = tokenizer(example["sentence1"], example["sentence2"], truncation=True, max_length=32,
        return_overflowing_tokens=True)
    sample_maping = result.pop("overflow_to_sample_mapping")

    for k, v in example.items():
      result[k] = [v[i] for i in sample_maping]

    return result

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
# 注意这里需要保留labels
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

在token化的过程中，截断与否都不会自动Padding。

In [78]:
# tokenized_datasets['train']
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [79]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 18525
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2051
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 8702
    })
})

In [58]:
len(tokenized_datasets['train'][1]['input_ids'])

27

In [80]:
# 前面已经remove，这里不再需要

tokenized_datasets = tokenized_datasets.remove_columns(['sentence1', 'sentence2','idx'])
# tokenized_datasets = tokenized_datasets.remove_columns(['overflow_to_sample_mapping'])
# tokenized_datasets = tokenized_datasets.rename_column('label','labels')  # 实践证明，这一行是不需要的
tokenized_datasets.set_format('torch')
print(tokenized_datasets['train'].column_names)


['label', 'input_ids', 'token_type_ids', 'attention_mask']


In [81]:
from torch.utils.data import DataLoader, Dataset
train_dataloader = DataLoader(tokenized_datasets['train'], shuffle=True, batch_size=8, collate_fn=data_collator)  # 通过这里的dataloader，每个batch的seq_len可能不同
eval_dataloader = DataLoader(tokenized_datasets['validation'], batch_size=8, collate_fn=data_collator)

# 查看一下train_dataloader的元素长啥样
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}
# batch



{'input_ids': torch.Size([8, 32]),
 'token_type_ids': torch.Size([8, 32]),
 'attention_mask': torch.Size([8, 32]),
 'labels': torch.Size([8])}

In [12]:
tokenized_datasets  # 经过上面的处理，它就可以直接丢进pytorch的Dataloader中了，跟pytorch中的Dataset格式已经一样了

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [82]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [84]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)
# batch

tensor(0.6202, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


In [85]:
from transformers import AdamW, get_scheduler

optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)  # num of batches * num of epochs
lr_scheduler = get_scheduler(
    'linear',
    optimizer=optimizer,  # scheduler是针对optimizer的lr的
    num_warmup_steps=0,
    num_training_steps=num_training_steps)
print(num_training_steps)


6948


In [86]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [89]:
from tqdm import tqdm
import torch.nn as nn

for epoch in range(num_epochs):
  running_loss = 0.0
  for i, batch in enumerate(tqdm(train_dataloader)):
      # 要在GPU上训练，需要把数据集都移动到GPU上：
      batch = {k:v.to(device) for k,v in batch.items()}
      loss = model(**batch).loss
      loss.backward()
      optimizer.step()
      lr_scheduler.step()
      optimizer.zero_grad()

      running_loss += loss.item()

      if i % 10 == 9:
        print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 10))
        running_loss = 0.0




  0%|          | 10/2316 [00:29<1:53:03,  2.94s/it]

[1,    10] loss: 0.617


  1%|          | 20/2316 [00:59<1:50:23,  2.88s/it]

[1,    20] loss: 0.605


  1%|▏         | 30/2316 [01:30<1:56:24,  3.06s/it]

...

[1,   680] loss: 0.692


 30%|██▉       | 690/2316 [34:21<1:20:40,  2.98s/it]

[1,   690] loss: 0.554


 30%|██▉       | 690/2316 [34:22<1:21:00,  2.99s/it]


KeyboardInterrupt: 

In [90]:
import evaluate

metric = evaluate.load("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.6962457337883959, 'f1': 0.8000000000000002}